# Introduction

- Douwe Maljers (15264424)
- Justin van Egmond (15011178)
- Noor Verbrugge (15168298)

Mentale gezondheid onder studenten is de afgelopen jaren een groeiend maatschappelijk thema geworden. De combinatie van allemaal verschillende omstandigheden, zoals prestatiedruk, sociale verwachtingen en onzekerheid over de toekomst maakt deze groep bijzonder kwetsbaar. In deze data-analyse richten we ons op het identificeren van factoren die samenhangen met mentale gezondheid, met als doel het onthullen van mogelijke risico’s en beschermende factoren binnen het studentenleven.

Op basis van enquêtegegevens van studenten wereldwijd, verkennen we de verbanden tussen mentale gezondheid en verschillende leefomstandigheden, gedragingen en gevoelens. Daarbij kijken we naar zowel interne als externe factoren die het welzijn kunnen beïnvloeden. De persectieven waaruit wij naar de mentale gezondheid gaan kijken zijn, social media gebruik, slaap, sport gewoonten ....

Wereldwijd is de populatie met mentale stoornissen significant gestegen. In de grafiek boven is per land de trend te zien. Vandaar dat dit onderzoek nog meer van belang is. Vrouwen en mannen zijn beide slachtoffer van deze chronische ziekte. 

In dit onderzoek worden meerdere standpunten onderbouwd aan de hand van data visualisaties, specifiek gericht op de invloeden op metale gezondheid onder studenten. Dit wordt gedaan met behulp van verschillende datasets van diverse groepen studenten. Tevens wordt een dataset van het Institute for Health Metrics and Evaluation gebruikt om de grootschaligheid van het wereldwijde probleem van mentale stoornissen en verminderde mentale gesteldheid in kaart te brengen.

In [2]:
import pandas as pd
import plotly.express as px
from ipywidgets import Dropdown, SelectionSlider, Layout, HBox, VBox, interactive_output
from IPython.display import display, HTML
import pycountry
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

# === Data inladen ===
csv_path = "datasets/IHME_mental_real.csv"
df2 = pd.read_csv(csv_path)

# === Landen omzetten naar ISO3-codes ===
def get_iso3(name):
    try:
        return pycountry.countries.lookup(name).alpha_3
    except LookupError:
        return None

df2["Country"] = df2["location"].apply(get_iso3)

df2["val_display"] = df2["val"]
df2.loc[df2["metric"] == "Percent", "val_display"] *= 100

# === Vaste schaal voor percentages ===
percent_max = df2.loc[df2["metric"] == "Percent", "val_display"].max()

year = 2015
sex = 'Both'
metric = 'Rate'

dff = df2[(df2["year"] == year) & (df2["sex"] == sex) & (df2["metric"] == metric)]
if dff.empty:
    print("Geen data beschikbaar voor deze selectie.")

if metric == "Percent":
    color_range = [0, percent_max]
else:
    color_range = [df2["val_display"].min(), df2["val_display"].max()]

fig = px.choropleth(
    dff,
    locations="Country",
    color="val_display",
    hover_name="location",
    color_continuous_scale="Reds",
    range_color=color_range,
    labels={"val_display": f"Prevalentie ({metric})"},
    title=f"Mental Disorders – {sex} – {metric} – {year}",
    width=1100,
    height=650
)
fig.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type="natural earth"
    ),
    margin=dict(l=0, r=0, t=50, b=0)
)
fig.show()

In [4]:
import pandas as pd
import plotly.graph_objects as go
import pycountry

# === Data inladen ===
csv_path = "datasets/IHME_mental_real.csv"
df = pd.read_csv(csv_path)

# === Landen naar ISO3 codes ===
def get_iso3(name):
    try:
        return pycountry.countries.lookup(name).alpha_3
    except LookupError:
        return None

df["iso_alpha"] = df["location"].apply(get_iso3)
df["val_display"] = df["val"]
df.loc[df["metric"] == "Percent", "val_display"] *= 100
percent_max = df.loc[df["metric"] == "Percent", "val_display"].max()

# === Parameters ===
sex = "Both"
metric = "Rate"
min_val = df["val_display"].min()
max_val = df["val_display"].max()

# === Unieke jaren ===
years = sorted(df["year"].unique())

# === Frames per jaar ===
frames = []
for year in years:
    dff = df[(df["year"] == year) & (df["sex"] == sex) & (df["metric"] == metric)]
    if dff.empty:
        continue
    frames.append(
        go.Frame(
            data=[
                go.Choropleth(
                    locations=dff["iso_alpha"],
                    z=dff["val_display"],
                    text=dff["location"],
                    colorscale="Reds",
                    zmin=min_val if metric != "Percent" else 0,
                    zmax=max_val if metric != "Percent" else percent_max,
                    colorbar_title=f"Prevalentie ({metric})",
                )
            ],
            name=str(year),
        )
    )

# === Initieel figuur voor eerste jaar ===
first_year = years[0]
df_first = df[(df["year"] == first_year) & (df["sex"] == sex) & (df["metric"] == metric)]

fig = go.Figure(
    data=[
        go.Choropleth(
            locations=df_first["iso_alpha"],
            z=df_first["val_display"],
            text=df_first["location"],
            colorscale="Reds",
            zmin=min_val if metric != "Percent" else 0,
            zmax=max_val if metric != "Percent" else percent_max,
            colorbar_title=f"Prevalentie ({metric})",
        )
    ],
    layout=go.Layout(
        title_text=f"Mental Disorders – {sex} – {metric} – {first_year}",
        geo=dict(showframe=False, showcoastlines=True, projection_type="natural earth"),
        sliders=[
            {
                "steps": [
                    {
                        "method": "animate",
                        "args": [[str(y)], {"mode": "immediate", "frame": {"duration": 800, "redraw": True}, "transition": {"duration": 0}}],
                        "label": str(y),
                    }
                    for y in years
                ],
                "transition": {"duration": 0},
                "x": 0.1,
                "len": 0.8,
                "xanchor": "left",
                "yanchor": "top",
                "pad": {"b": 10, "t": 50},
                "currentvalue": {"prefix": "Jaar: ", "font": {"size": 16}},
            }
        ],
        updatemenus=[
            {
                "type": "buttons",
                "showactive": False,
                "x": 0.05,
                "y": 1.12,
                "xanchor": "right",
                "yanchor": "top",
                "pad": {"r": 10, "t": 70},
                "buttons": [
                    {
                        "label": "▶️",
                        "method": "animate",
                        "args": [None, {"frame": {"duration": 800, "redraw": True}, "fromcurrent": True}],
                    },
                    {
                        "label": "⏸️",
                        "method": "animate",
                        "args": [[None], {"mode": "immediate", "frame": {"duration": 0}, "transition": {"duration": 0}}],
                    },
                ],
            }
        ],
    ),
    frames=frames,
)

fig.show()
